# Limpeza do ambiente do SageMaker

<div style="border: 4px solid coral; text-align: center; margin: auto;">
    <p style="text-align: center; margin: auto;">
        <b>Este notebook remove todos os recursos criados por todos os notebooks que você executou em seu ambiente.</b>
    </p>
</div>

❗ As seguintes células de código irão:
- deletar permanentemente o projeto ou projetos que você provisionou em seu ambiente Studio
- deletar permanentemente grupos de features
- deletar permanentemente buckets S3 provisionados pelo projeto
- deletar permanentemente objetos nos buckets S3 sob prefixos relacionados ao projeto
- deletar permanentemente endpoints de inferência

<div class="alert alert-info"> ❗ Células de código que listam projetos, grupos de features e endpoints mostram todos os recursos do SageMaker na sua conta da AWS, mesmo aqueles criados <b>fora deste treinamento</b>. Verifique novamente os nomes dos recursos a serem deletados para não excluir seus próprios recursos.
</div>

<div class="alert alert-info"> 💡 <strong> Este notebook irá deletar permanentemente recursos na sua conta da AWS. Por favor, verifique novamente os nomes dos recursos a serem deletados! </strong>
</div>

<div class="alert alert-info">Você não precisa executar este notebook se estiver usando uma conta AWS provisionada pela AWS em um workshop liderado por instrutor.</div>

<div class="alert alert-info"> Certifique-se de estar usando o kernel <code>Python 3</code> no JupyterLab para este notebook.</div>

In [2]:
import sagemaker
import boto3
import time
import json
import os

sagemaker.config INFO - Not applying SDK defaults from location: /etc/xdg/sagemaker/config.yaml
sagemaker.config INFO - Not applying SDK defaults from location: /home/sagemaker-user/.config/sagemaker/config.yaml


In [3]:
%store -r 

%store

try:
    initialized
except NameError:
    print("+++++++++++++++++++++++++++++++++++++++++++++++++")
    print("[ERRO] VOCÊ PRECISA EXECUTAR O CADERNO 00-inicie-aqui")
    print("+++++++++++++++++++++++++++++++++++++++++++++++++")


Stored variables and their in-db values:
bucket_name                         -> 'sagemaker-us-east-1-906545278380'
bucket_prefix                       -> 'from-idea-to-prod/xgboost'
dataset_file_local_path             -> 'data/bank-additional/bank-additional-full.csv'
domain_id                           -> 'd-1dklekwofndh'
experiment_name                     -> 'from-idea-to-prod-experiment-25-20-38-00'
initialized                         -> True
input_s3_url                        -> 's3://sagemaker-us-east-1-906545278380/from-idea-t
ml_flow_arn                         -> 'arn:aws:sagemaker:us-east-1:906545278380:mlflow-t
region                              -> 'us-east-1'
sm_role                             -> 'arn:aws:iam::906545278380:role/service-role/Amazo
target_col                          -> 'y'
user_profile_name                   -> None


In [4]:
sm = boto3.client("sagemaker")
s3 = boto3.resource('s3')

## Delete projetos

In [ ]:
# Obter todos os projetos criados pelo domínio atual
projects = [
    {"ProjectName": p["ProjectName"], "ProjectId": p["ProjectId"]} for p in sm.list_projects(MaxResults=100, SortBy="CreationTime")["ProjectSummaryList"] 
        if sm.describe_project(ProjectName=p["ProjectName"])["CreatedBy"]["DomainId"] == domain_id and p["ProjectStatus"] == "CreateCompleted"
]

print(f"Esses projetos foram criados pelo domínio {domain_id}: {json.dumps(projects, indent=2)}")


In [ ]:
# Selecionar projetos a serem deletados
projects_to_delete = []

for p in projects:
    print(f"Você tem certeza de que deseja deletar este projeto: {p['ProjectName']}? (y/n)")
    choice = input()
    if choice == 'y':
        projects_to_delete.append(p)
        
print(f"***************************************")
print(f"Os seguintes projetos serão deletados:\n{json.dumps(projects_to_delete, indent=2)}")
print(f"***************************************")


In [ ]:
for p in projects_to_delete:
    try:
        print(f"Deletando projeto {p['ProjectName']}: {sm.delete_project(ProjectName=p['ProjectName'])}")
    except Exception:
        pass


## Excluir programações de monitoramento
Você deve excluir os agendamentos de monitoramento para poder remover com êxito os endpoints implantados pelos projetos MLOps.

In [ ]:
def delete_mon_schedules(endpoint_name, sm_client):
    print(f"Deletar agendamentos de monitoramento para o endpoint: {endpoint_name}")
    for s in sm_client.list_monitoring_schedules(EndpointName=endpoint_name)["MonitoringScheduleSummaries"]:
        print(f"Deletar agendamento de monitoramento: {s['MonitoringScheduleName']}")
        r = sm_client.delete_monitoring_schedule(MonitoringScheduleName=s['MonitoringScheduleName'])
        print(r)


In [ ]:
for p in projects_to_delete:
    delete_mon_schedules(f"{p['ProjectName']}-staging", sm)
    delete_mon_schedules(f"{p['ProjectName']}-prod", sm)

## Excluir pilhas do CloudFormation
Esta seção exclui as pilhas do AWS CloudFormation que os projetos criaram

In [ ]:
cfn = boto3.client("cloudformation")

for p in projects_to_delete:
    for s in [
            f"sagemaker-{p['ProjectName']}-{p['ProjectId']}-deploy-staging",
            f"sagemaker-{p['ProjectName']}-{p['ProjectId']}-deploy-prod"
            ]:
        try:
            print(f"Delete CloudFormation stack: {s}")
            r = cfn.delete_stack(StackName=s)
            print(r)
            time.sleep(180)
        except Exception as e:
            print(f"Exception in deleting {s}:{e}")
            pass

## Delete feature groups

In [ ]:
feature_groups = sm.list_feature_groups(
    FeatureGroupStatusEquals="Created", 
    SortOrder="Descending", 
    SortBy="CreationTime"
)["FeatureGroupSummaries"]

In [ ]:
feature_groups

In [ ]:
feature_groups_to_delete = []

for fg in feature_groups:
    print(f"Você tem certeza de que deseja deletar este grupo de features: {fg['FeatureGroupName']}? (y/n)")
    choice = input()
    if choice == 'y':
        feature_groups_to_delete.append(fg["FeatureGroupName"])
        
print(f"********************************************")
print(f"Os seguintes grupos de features serão deletados:\n{json.dumps(feature_groups_to_delete, indent=2)}")
print(f"********************************************")


In [ ]:
def delete_offline_store(feature_group_name: str):
    try:
        offline_store_config = sm.describe_feature_group(FeatureGroupName=feature_group_name)['OfflineStoreConfig']

    except Exception:
        print(f'O grupo de features: {feature_group_name} NÃO possui um armazenamento offline!')
        return
    
    offline_store_s3_uri = offline_store_config['S3StorageConfig']['ResolvedOutputS3Uri']
    print(f"Todos os objetos do armazenamento de features sob {offline_store_s3_uri} serão deletados!")
    print("Você tem certeza de que deseja deletar esses objetos? (y/n)")
    
    choice = input()
    if choice == 'y':
        !aws s3 rm {offline_store_s3_uri} --recursive


<div class="alert alert-info"> 💡 <strong>A próxima célula de código irá deletar os grupos de features selecionados!</strong>
</div>

In [ ]:
for fg in feature_groups_to_delete:
    print(f"Deletando o grupo de características: {fg}")
    delete_offline_store(fg)
    sm.delete_feature_group(FeatureGroupName=fg)


## Excluir os buckets S3 provisionados pelo projeto

<div class="alert alert-info"> 💡 <strong>A próxima célula de código irá deletar todos os buckets S3 criados por um projeto!</strong> Se você estiver em um workshop liderado por instrutor e usando uma conta AWS provisionada, pode não ser possível deletar nenhum bucket S3. Você pode parar aqui.
</div>

In [ ]:
print(f"*****************************************************")
print(f"Os seguintes buckets S3 serão removidos permanentemente!")
print(f"*****************************************************")
for p in projects_to_delete:
    print(f"sagemaker-project-{p['ProjectId']}")


In [ ]:
for p in projects_to_delete:
    !aws s3 rb s3://sagemaker-project-{p['ProjectId']} --force 

## Remove endpoints de inferencia

In [ ]:
endpoints = sm.list_endpoints()["Endpoints"]

In [ ]:

for ep in endpoints:
    print(f"Você tem certeza de que deseja deletar este endpoint: {ep['EndpointName']}? (y/n)")
    choice = input()
    if choice == 'y':
        endpoints_to_delete.append(ep['EndpointName'])
        
print(f"*********** ESTES ENDPOINTS SERÃO DELETADOS ***********")
print('\n'.join(endpoints_to_delete))
print(f"**")
*****************************************************")

In [ ]:
for ep in endpoints_to_delete:
    try:
        delete_mon_schedules(ep, sm)
        time.sleep(10)
        print(f"Deletando o endpoint: {ep}: {sm.delete_endpoint(EndpointName=ep)}")
    except Exception as e:
        print(f"Exceção ao deletar {ep}: {e}")
        pass


## Remova os objetos relacionados ao projeto do bucket de dados do SageMaker S3

<div class="alert alert-info"> 💡 <strong>A próxima célula de código irá deletar todos os objetos sob os prefixos S3 especificados!</strong>
</div>

In [ ]:
prefixes_to_delete = [
    bucket_prefix,
    #s3_fs_query_output_prefix
]

In [ ]:
print(f"************************************************************************")
print(f"Todos os objetos sob os seguintes prefixos S3 serão removidos permanentemente!")
print(f"************************************************************************")
for p in prefixes_to_delete:
    print(f"{bucket_name}/{p}")


<div class="alert alert-info"> ❗ <strong> Double check the S3 prefix! All data under this prefix will be permanently deleted!</strong>
</div>

In [ ]:
# Descomente o comando rm
for p in prefixes_to_delete:
    # !aws s3 rm s3://{bucket_name}/{p} --recursive
    pass


## Delete servidor MLflow
Uma vez criado, um servidor de rastreamento MLflow incorrerá custos até que você o delete ou pare. A cobrança pelos servidores de rastreamento é baseada na duração em que os servidores estiveram em funcionamento, no tamanho selecionado e na quantidade de dados registrados nos servidores de rastreamento.

Você deve deletar todos os servidores de rastreamento MLFlow criados manualmente para evitar custos adicionais. 

<div class="alert alert-info">Você não precisa deletar servidores de rastreamento MLflow e executar as células correspondentes se não criou nenhum servidor de rastreamento manualmente via UX ou API.</div>

In [15]:
# Encontrar todos os servidores de rastreamento MLflow ativos no domínio
tracking_servers = sm.list_mlflow_tracking_servers(
    TrackingServerStatus='Created',
)['TrackingServerSummaries']

ts_to_delete = []

for ts in tracking_servers:
    print(f"Você tem certeza de que deseja deletar este servidor de rastreamento MLflow: {ts['TrackingServerName']}? (y/n)")
    choice = input()
    if choice == 'y':
        ts_to_delete.append(ts['TrackingServerName'])

print(f"*********** ESTES SERVIDORES DE RASTREAMENTO MLFLOW SERÃO DELETADOS ***********")
print('\n'.join(ts_to_delete))
print(f"**********************************************************************")


Are you sure you want to delete this MLflow tracking server: mlflow-server-d-1dklekwofndh? (y/n)


 y


*********** THESE MLFLOW TRACKING SERVERS WILL BE DELETED ***********
mlflow-server-d-1dklekwofndh
**********************************************************************


In [16]:
for ts in ts_to_delete:
    try:
        print(f"Deletando o servidor de rastreamento MLflow: {ts}: {sm.delete_mlflow_tracking_server(ts)}")
    except Exception as e:
        print(f"Exceção ao deletar {ts}: {e}")
        pass


Deleting the MLflow tracking server: mlflow-server-d-1dklekwofndh


# Desligar o kernel
=

In [ ]:
%%html

<p><b>Shutting down your kernel for this notebook to release resources.</b></p>
<button class="sm-command-button" data-commandlinker-command="kernelmenu:shutdown" style="display:none;">Shutdown Kernel</button>
        
<script>
try {
    els = document.getElementsByClassName("sm-command-button");
    els[0].click();
}
catch(err) {
    // NoOp
}    
</script>